# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [1]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [3]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train_copy = X_train.copy()
    X_val_copy = X_val.copy()
    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
        # standardizzo i dati
            scaler = StandardScaler()
            X_train_copy=scaler.fit_transform(X_train_copy)
            X_val_copy=scaler.transform(X_val_copy)
        # applico la pca 
        pca=PCA(n_components=hyperparams['pca_components'])
        X_train_copy=pca.fit_transform(X_train_copy)
        X_val_copy=pca.transform(X_val_copy)

    if hyperparams['data_standardize']:
        # standardizzo i dati
        scaler = StandardScaler()
        X_train_copy=scaler.fit_transform(X_train_copy)
        X_val_copy=scaler.transform(X_val_copy)

    # Aggiunge il termine costante ai dati
    # aggiungo un vettore costante di uno
    X_train_copy=np.c_[np.ones((X_train_copy.shape[0], 1)), X_train_copy]
    X_val_copy=np.c_[np.ones((X_val_copy.shape[0], 1)), X_val_copy]

    
    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
         #se richiesto applico la regolarizzazione L2
        lambda_ = hyperparams['reg_lambda']
        I = np.eye(X_train_copy.shape[1])
        I[0, 0] = 0  #non regolarizzo il parametro costante aggiunto
        
        X_trasposta = X_train_copy.T  
        inversa = np.linalg.inv(X_trasposta @ X_train_copy + lambda_ * I) 
        parametri=(inversa @ X_trasposta) @ y_train
    else:
        X_trasposta = X_train_copy.T  
        inversa = np.linalg.inv(X_trasposta @ X_train_copy) 
        parametri=(inversa @ X_trasposta) @ y_train

    
    # Calcolo predizioni
    y_train_pred = X_train_copy @ parametri
    y_val_pred = X_val_copy @ parametri
    

    # Calcola il MAE
    MAE_val=np.mean(np.abs(y_val-y_val_pred))
    MAE_train=np.mean(np.abs(y_train-y_train_pred))

    return MAE_val, MAE_train, parametri

# Dividi il dataset in training e test set
train_fraction = 0.8  
test_fraction = 0.2

num_train = int(train_fraction * X.shape[0])
X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]

print("Training set:", X_train.shape)
print("Test set:", X_test.shape)

# Dividi il training set in training set effettivo e validation set
train_fraction_effettivo = 0.8 * train_fraction 
validation_fraction = 0.2 * train_fraction  
num_train = int(train_fraction_effettivo * X.shape[0])
num_validation = int(validation_fraction * X.shape[0])

X_train_eff = X[:num_train]
y_train_eff = y[:num_train]

X_validation = X[num_train:num_train+num_validation]
y_validation = y[num_train:num_train+num_validation]

print("Training set effettivo:", X_train_eff.shape)
print("Validation set:", X_validation.shape)

# Trova la configurazione di iperparametri migliore
best_config = None
best_val_mae = float('inf')
best_train_mae = None

for values in configs: #scelgo la configurazione con il valore minore di mae sulla validation
    mae_val, mae_train, _ = pipeline(X_train_eff, y_train_eff, X_validation, y_validation, values)
    if mae_val < best_val_mae:
        best_val_mae = mae_val 
        best_train_mae = mae_train
        best_config = values
        
print("Miglior configurazione trovata:")
print(best_config)
print("MAE Validation:", best_val_mae)
print("MAE Training:", best_train_mae)

# Riallena il modello sul training set completo (training set + validation set)
_, _, best_parametri = pipeline(X_train, y_train, X_test, y_test, best_config)

X_test_copy = X_test.copy()
X_train_copy = X_train.copy()

if best_config['use_pca']:
    if best_config['pca_standardize']:
        X_train_copy = scaler.fit_transform(X_train_copy)
        X_test_copy = scaler.transform(X_test_copy)
    pca = PCA(n_components=best_config['pca_components'])
    X_train_copy = pca.fit_transform(X_train_copy)
    X_test_copy = pca.transform(X_test_copy)

if best_config['data_standardize']:
    scaler = StandardScaler()
    X_train_copy = scaler.fit_transform(X_train_copy)
    X_test_copy = scaler.transform(X_test_copy)
    
X_train_copy = np.c_[np.ones((X_train_copy.shape[0], 1)), X_train_copy]
X_test_copy = np.c_[np.ones((X_test_copy.shape[0], 1)), X_test_copy]

# Calcolo le predizioni sul test set
y_test_pred = X_test_copy @ best_parametri

# Calcola il MAE sul test set
mae_test = np.mean(np.abs(y_test - y_test_pred))

print("MAE Test: ", mae_test)

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Training set: (3918, 11)
Test set: (980, 11)
Training set effettivo: (3134, 11)
Validation set: (783, 11)
Miglior configurazione trovata:
{'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE Validation: 0.5735025699479851
MAE Training: 0.6061022582434107
MAE Test:  0.5481392209725904
